In [2]:
# !wget https://object.pouta.csc.fi/OPUS-QED/v2.0a/raw/en.zip
# !wget https://object.pouta.csc.fi/OPUS-QED/v2.0a/raw/ms.zip
# !unzip -o -qq en.zip
# !unzip -o -qq ms.zip

In [4]:
# !wget wget https://object.pouta.csc.fi/OPUS-QED/v2.0a/xml/en-ms.xml.gz
# !gzip -d en-ms.xml.gz

In [6]:
from bs4 import BeautifulSoup
from tqdm import tqdm

with open('en-ms.xml') as fopen:
    xml = fopen.read().split('\n')
    
mapping, ids = {}, []
enable = False
for row in tqdm(xml):
    if '<linkGrp targType' in row:
        s = BeautifulSoup(row)
        r = 'QED/raw/' + s.findAll('linkgrp')[0].get('fromdoc').replace('.gz','')
        l = 'QED/raw/' + s.findAll('linkgrp')[0].get('todoc').replace('.gz','')
        enable = True
    if '<link id' in row and enable:
        s = BeautifulSoup(row)
        t = s.findAll('link')[0].get('xtargets')
        t = list(filter(None, t.split(';')))
        if len(t) != 2:
            continue
        ids.append([t[0].split(), t[1].split()])
    if '</linkGrp>' in row:
        label = f'{l} <> {r}'
        mapping[label] = ids
        enable = False
        ids = []

100%|██████████| 108998/108998 [00:25<00:00, 4300.02it/s]


In [7]:
results = []
for file in tqdm(mapping.keys()):
    l, r = file.split(' <> ')
    ids = mapping[file]
    try:
        with open(l) as fopen:
            left = fopen.read()
        soup_left = BeautifulSoup(left, 'html.parser')
        with open(r) as fopen:
            right = fopen.read()
        soup_right = BeautifulSoup(right, 'html.parser')
        left_dict = {}
        for s in soup_left.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            left_dict[i] = text
        right_dict = {}
        for s in soup_right.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            right_dict[i] = text
        combined = []
        for i in ids:
            try:
                r = ' '.join([right_dict[k] for k in i[0]])
                l = ' '.join([left_dict[k] for k in i[1]])
                combined.append((l, r))
            except:
                pass
        results.extend(combined)
    except Exception as e:
        print(e)
        pass

100%|██████████| 812/812 [00:46<00:00, 17.38it/s]


In [8]:
len(results)

79871

In [9]:
results[:20]

[('\n\nKita diminta untuk mendarab 65 dengan 1. \n    \n',
  "\n\nWe're asked to multiply 65 times 1. \n    \n"),
 ('\n\nSecara literal, kita hanya perlu darabkan 65--kita boleh tuliskannya \n    \n\nsebagai tanda darab atau kita boleh tuliskan sebagai titik \n    \n\nmacam ini--tapi ia bermaksud 65 darab 1. \n    \n',
  '\n\nSo literally, we just need to multiply 65, \n    \n\nand we could write it as a times sign like that or we could write it as a dot like that \n    \n\nbut this means 65 times 1. \n    \n'),
 ('\n\nAda dua cara untuk mentafsir ini. \n    \n',
  "\n\nAnd there's two ways to interpret this. \n    \n"),
 ('\n\nKamu boleh lihat ini sebagai nombor 65 sebanyak satu kali atau kamu \n    \n\nboleh lihatnya sebagai nombor 1 sebanyak enam puluh lima \n    \n\nkali. \n    \n',
  '\n\nYou could view this as the number 65 one time \n    \n\nor you could view this as the number 1 sixty-five times, all added up. \n    \n'),
 ('\n\nMana-mana cara pun, jika kamu ada 65, ini secara 

In [10]:
import json

with open('qed-ms-en.json', 'w') as fopen:
    json.dump(results, fopen)

In [11]:
!rm -rf QED
!rm -rf ms.zip en.zip
!rm -rf ms.zip en.zip en-ms.xml.gz en-ms.xml